In [1]:
####Visualizations
#### Importing dependencies 
import numpy as np
import pandas as pa
import seaborn as se
import matplotlib.pyplot as plt
from IPython.display import HTML

In [2]:
##### Data preparation for the Prediction.


ball_by_ball=pa.read_csv("./train/train/ball_by_ball_data.csv")
player_roster=pa.read_csv("./train/train/player_rosters.csv")
key_teams=pa.read_csv('./train/train/key_teams.csv')
match_data=pa.read_csv('./train/train/match_data.csv')

In [3]:
##### Now take ball_by_ball data for preprocessing.
##### Grouping the data such a way that we get the runs scored and wickets taken by each player for each match.
##### One more constraint needed for grouping is innings based on innings we are grouping and splitting into two datasets.


batsman_score=ball_by_ball[['match_id','inning','batting_team','bowling_team','batsman','batsman_id','batsman_runs']]
batsman_score.head()
batsman_score=batsman_score.groupby(['match_id','inning','batting_team','bowling_team','batsman','batsman_id'], as_index=False)['batsman_runs'].sum()
batsman_score_1st_inn=batsman_score[batsman_score['inning']==1]
batsman_score_2nd_inn=batsman_score[batsman_score['inning']==2]


In [11]:
wickets_taken = ball_by_ball[['match_id','inning','batting_team','bowling_team','bowler','bowler_id','dismissal_kind']]

In [12]:
wickets_taken.fillna(0,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [6]:
wickets_taken.head()

,match_id,inning,batting_team,bowling_team,bowler,bowler_id,dismissal_kind
0,2017_1,1,SRH,RCB,Tymal Mills,621,0
1,2017_1,1,SRH,RCB,Tymal Mills,621,0
2,2017_1,1,SRH,RCB,Tymal Mills,621,0
3,2017_1,1,SRH,RCB,Tymal Mills,621,0
4,2017_1,1,SRH,RCB,Tymal Mills,621,0


In [8]:
wickets_taken['dismissal_kind'].unique()

array([0, 'caught', 'bowled', 'run out', 'lbw', 'caught and bowled',
       'stumped', 'retired hurt', 'hit wicket', 'obstructing the field'], dtype=object)

In [9]:
dismissal_counted_for_bowlers=['caught', 'bowled', 'lbw', 'caught and bowled','stumped','hit wicket']

In [10]:
mapping={'caught':1, 'bowled':1, 'lbw':1, 'caught and bowled':1,'stumped':1,'hit wicket':1,'run out':0,'retired hurt':0,'obstructing the field':0}

In [15]:
wickets_taken=wickets_taken.replace({'dismissal_kind':mapping})

In [18]:
wickets_taken=wickets_taken.groupby(['match_id','inning','batting_team','bowling_team','bowler','bowler_id'],as_index=False)['dismissal_kind'].sum()

In [21]:
wickets_taken_1st_inn=wickets_taken[wickets_taken['inning']==1]
wickets_taken_2nd_inn=wickets_taken[wickets_taken['inning']==2]

In [28]:
np.count_nonzero(wickets_taken_1st_inn['bowler_id'].unique())

330

In [31]:
np.count_nonzero(wickets_taken_2nd_inn['bowler_id'].unique())

323

In [36]:
player_ids=player_roster['player_id'].unique()

In [40]:
player_ids.size

681

In [41]:
pa.DataFrame(columns=player_ids)

,1,2,3,4,5,6,7,8,9,10,...,687,688,689,690,691,692,693,694,695,696


In [43]:
##### implementing decision tree for the bowler taking in 1st innings

##### import decision tree dependencies.

from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeRegressor
